In [1]:
# Dependencies
import requests
from config_env import g_key
import time
import numpy as np
import pandas as pd
import json
import os
from IPython.core.display import display, HTML
import pymongo
from pymongo import MongoClient

# Store a search term
query = "Venezuela"

# Search for articles published between a begin and end date
begin_date = "2014-01-01"
end_date = "20190101"
offset = '&offset='

url = f"https://content.guardianapis.com/search?section=\
world&page-size=100&from-date={begin_date}&q={query}&api-key={g_key}&page="


In [42]:
output_list = []
# Loop through the 2000 request limit and grab just the URL,
# headline, article type and published date
for i in range(1, 2000):
    query_url = url + str(i)
    try:
        articles = requests.get(query_url).json()
        if articles['response']['status'] == 'ok': # Maybe should use a While loop...
            articles_list = [article for article in articles["response"]["results"]]
            for item in articles_list:
                list_dict = {}
                list_dict["url"] = item["apiUrl"]
                list_dict["headline"] = item["webTitle"]
                list_dict["type"] = item["type"]
                list_dict["pub_date"] = item["webPublicationDate"]
                output_list.append(list_dict)
    except:
        pass


In [40]:
print(articles)

{'response': {'status': 'ok', 'userTier': 'developer', 'total': 610, 'startIndex': 1, 'pageSize': 100, 'currentPage': 1, 'pages': 7, 'orderBy': 'relevance', 'results': [{'id': 'world/2019/jan/30/venezuela-maduro-accuses-us-of-trying-to-get-hands-on-oil-donald-trump', 'type': 'article', 'sectionId': 'world', 'sectionName': 'World news', 'webPublicationDate': '2019-01-31T05:07:31Z', 'webTitle': "Venezuela: Juan Guaidó says Venezuela opposition 'has met military'", 'webUrl': 'https://www.theguardian.com/world/2019/jan/30/venezuela-maduro-accuses-us-of-trying-to-get-hands-on-oil-donald-trump', 'apiUrl': 'https://content.guardianapis.com/world/2019/jan/30/venezuela-maduro-accuses-us-of-trying-to-get-hands-on-oil-donald-trump', 'isHosted': False, 'pillarId': 'pillar/news', 'pillarName': 'News'}, {'id': 'world/2019/feb/17/proposal-to-end-the-crisis-in-venezuela', 'type': 'article', 'sectionId': 'world', 'sectionName': 'World news', 'webPublicationDate': '2019-02-17T17:54:30Z', 'webTitle': 'Pr

In [43]:
# Establish a connection to Mongo and make sure it works
print('Mongo version', pymongo.__version__)
client = MongoClient('localhost', 27017)
db = client.venezuela

Mongo version 3.7.2


In [44]:
# Insert all the news metadata to the mongodb collection
db.guardian_news.insert_many(output_list)

In [65]:
# Only pull back news articles for full text
news = db.guardian_news.find({'type':'article'})

In [66]:
# Get a list of URLs to scrape
urls = []
for item in news:
    urls.append(item['url'])

In [67]:
len(urls)

682